## This  is a notebook

In [18]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_absolute_error

In [2]:
properties = pd.read_csv('../data/properties_2016.csv')
train_data = pd.read_csv('../data/train_2016_v2.csv')
template_submission = pd.read_csv('../data/sample_submission.csv')

C:\Users\roel\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Preprocessing

- merge data frames
- remove crappy features (missing > .75)
- make sure the types make sense


we have data of houses that have been sold, those have a logerror. This logerror is our train_y. Split data set on logerror / no logerror. The other is test?



In [3]:
merged_df = pd.merge(train_data, properties,on='parcelid')

In [4]:
temp_df = merged_df[['bathroomcnt', 'bedroomcnt', 'longitude', 'latitude','logerror']].dropna()
properties_filtered = properties[['bathroomcnt', 'bedroomcnt', 'longitude', 'latitude']]
properties_filtered = properties_filtered.fillna(properties_filtered.mean())
x_train = temp_df[['bathroomcnt', 'bedroomcnt', 'longitude', 'latitude']]
y_train = temp_df.logerror
x_submission = properties_filtered

In [6]:
#impute grouped on unitcnt, buildingqualitytypeid, buildingclasstypeid, architecturalstyletypeid

Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildin

## Feature engineering

In [7]:
feature_engineering = Imputer(strategy="median", axis = 0) 


## Model

In [20]:
model = svm.SVR()


## Pipeline

In [21]:
pipe = Pipeline([('feature_engineering', feature_engineering),
                 ('model', model)
                 ]
)

In [ ]:
# Fit model
pipe.fit(x_train, y_train)

In [12]:
out_submission = pipe.predict(x_submission)

In [13]:
out_formatted = template_submission
for col in out_formatted.columns.values:
    if col == 'ParcelId':
        pass
    else:
        out_formatted[col] = out_submission
out_formatted.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,-0.000533,-0.000533,-0.000533,-0.000533,-0.000533,-0.000533
1,10759547,-0.000557,-0.000557,-0.000557,-0.000557,-0.000557,-0.000557
2,10843547,-0.000955,-0.000955,-0.000955,-0.000955,-0.000955,-0.000955
3,10859147,-0.000653,-0.000653,-0.000653,-0.000653,-0.000653,-0.000653
4,10879947,-0.000604,-0.000604,-0.000604,-0.000604,-0.000604,-0.000604


In [11]:
out_formatted.to_csv('../data/submission_1_JFR', index=False)